# Google Trends Local Deals Restuarants and Food Scripts 


In [6]:
from datetime import date
import pandas as pd
import pytrends
from pytrends.request import TrendReq
import datetime 

In [144]:
numyears = 1
numdays = 7
numweeks = 52
total_time_range = numdays * numweeks * numyears

end_date= date.today()
today_date = date.today()
end_date = today_date
begin_date = end_date - datetime.timedelta(days = total_time_range-7)

user_timeframe = begin_date.strftime('%Y-%m-%d')+' '+end_date.strftime('%Y-%m-%d')
print(user_timeframe)

2019-03-19 2020-03-10


In [145]:
# initiate API 
pytrend =  TrendReq()

In [146]:
# perform initial search for related queries 
pytrend.build_payload(kw_list=['food', 'best', 'restaurants', 'cafe', 'vegan'], geo = 'GB', timeframe = user_timeframe, cat = 71)
related_queries_dict = pytrend.related_queries()
related_topics_dict = pytrend.related_topics()

In [57]:
related_queries_dict

{'food': {'top':                      query  value
  0             food near me    100
  1                 bbc food     84
  2                good food     78
  3            delivery food     55
  4            bbc good food     52
  5             chinese food     37
  6             food recipes     37
  7   marks and spencer food     31
  8                best food     31
  9                 m&s food     27
  10               fast food     27
  11              vegan food     27
  12             indian food     24
  13              party food     22
  14         food restaurant     21
  15              order food     19
  16              tesco food     19
  17             street food     18
  18               thai food     17
  19               asda food     15
  20              food ideas     14
  21               free food     14
  22               food shop     13
  23   delivery food near me     13
  24             food places     12,
  'rising':                            query  va

In [88]:

def get_data(dicti, key):
    
    df1 = related_queries_dict.get(key).get('rising')
    df2 = related_queries_dict.get(key).get('top')
    
    df1 = pd.DataFrame(df1, columns=['query', 'value'])
    df2 = pd.DataFrame(df2, columns=['query', 'value'])
    
    
    return df1, df2

In [126]:
best_rising, best_top = get_data(related_queries_dict, key='best')
food_rising, food_top = get_data(related_queries_dict, key='food')
rest_rising, rest_top = get_data(related_queries_dict, key='restaurants')
cafe_rising, cafe_top = get_data(related_queries_dict, key='cafe')
vega_rising, vega_top = get_data(related_queries_dict, key='vegan')


In [108]:
def prep_data(df, key):
    
    searches = df[key].to_list()
    
    groupkeywords = list(zip(*[iter(searches)]*1))
    groupkeywords = [list(x) for x in groupkeywords]
    
    return groupkeywords, searches

In [136]:
best_risng_kws, searches_best = prep_data(best_rising, key='query')
food_risng_kws, searches_food = prep_data(food_rising, key='query')
rest_risng_kws, searches_rest = prep_data(rest_rising, key='query')
cafe_risng_kws, searches_cafe = prep_data(cafe_rising, key='query')
vega_risng_kws, searches_vega = prep_data(vega_rising, key='query')

In [149]:
def slope(xone, yone, xtwo, ytwo):
    return (ytwo-yone)/(xtwo-xone)

In [158]:
def perform_trending_search(groupkeywords, searches, filename):
    
    dicti = {}
    
    i = 1
    
    for trending in groupkeywords:
        pytrend.build_payload(trending, timeframe = user_timeframe, geo = 'GB', cat = 71)
        dicti[i] = pytrend.interest_over_time()
        i+=1
    
    result = pd.concat(dicti, axis=1)
    result.columns = result.columns.droplevel(0)
    result = result.drop('isPartial', axis = 1)
    result.reset_index(level=0, inplace=True)
    result = pd.melt(result, id_vars='date', value_vars=searches)
    
    
    
    keyword = result['value'].loc[[0]][0]
    trenddata = trenddata.ix[1:]
    trenddata['keyword'] = keyword
    trenddata.rename(columns={'values': 'trends'}, inplace=True)
    trenddata['trends'] = pd.to_numeric(trenddata['trends'], errors='coerce')
    trenddata['date'] = trenddata['date'].str.extract('(^[0-9]{4}\-[0-9]{2}\-[0-9]{2}) \-.*')
    trenddata = trenddata.dropna()
    trenddata['date'] = pd.to_datetime(trenddata['date'])
    trenddata['year'] = pd.DatetimeIndex(trenddata['date']).year
    trenddata['month'] = pd.DatetimeIndex(trenddata['date']).month
    trenddata['day'] = pd.DatetimeIndex(trenddata['date']).day
 
    maxyear = trenddata['year'].max()
    grouped = trenddata.groupby(['year']).mean() 
    
    slope = slope_formula(1,float(grouped.loc[grouped.index==maxyear-2]['trends']),
                          2,float(grouped.loc[grouped.index==maxyear-1]['trends']))
    keywordlist = keywordlist.append({'keyword':keyword,'slope':slope}, ignore_index=True)
    
    save_file = keywordlist.to_csv(filename)
    
    return keywordlist



In [159]:
# have to split and run in different cells or google will block the request
perform_trending_search(groupkeywords=best_risng_kws, searches=searches_best, filename='RisingLocalDealsBest.csv')

UnboundLocalError: local variable 'trenddata' referenced before assignment

In [138]:
# have to split and run in different cells or google will block the request
perform_trending_search(groupkeywords=best_risng_kws, searches=searches_best, filename='RisingLocalDealsBest.csv')
perform_trending_search(groupkeywords=rest_risng_kws, searches=searches_rest, filename='RisingLocalDealsRest.csv')
perform_trending_search(groupkeywords=food_risng_kws, searches=searches_food, filename='RisingLocalDealsFood.csv')

,date,variable,value
0,2019-03-24,food hub code,0
1,2019-03-31,food hub code,0
2,2019-04-07,food hub code,22
3,2019-04-14,food hub code,0
4,2019-04-21,food hub code,0
...,...,...,...
1245,2020-02-02,asda food shop,65
1246,2020-02-09,asda food shop,58
1247,2020-02-16,asda food shop,67
1248,2020-02-23,asda food shop,70


In [139]:
perform_trending_search(groupkeywords=cafe_risng_kws, searches=searches_cafe, filename='RisingLocalDealsCafe.csv')
perform_trending_search(groupkeywords=vega_risng_kws, searches=searches_vega, filename='RisingLocalDealsVega.csv')

,date,variable,value
0,2019-03-24,greggs vegan steak bake,0
1,2019-03-31,greggs vegan steak bake,0
2,2019-04-07,greggs vegan steak bake,0
3,2019-04-14,greggs vegan steak bake,0
4,2019-04-21,greggs vegan steak bake,0
...,...,...,...
1245,2020-02-02,vegan burger,30
1246,2020-02-09,vegan burger,22
1247,2020-02-16,vegan burger,23
1248,2020-02-23,vegan burger,23


In [147]:
import time
import os
from random import randint
